In [1]:
# Authors: M. Riley Owens (GitHub: mrileyowens) and Jane R. Rigby (GitHub: janerigby)

# This file is a total conversion of a previous file (Sunburst_NB_continuum_subtraction_v2.ipynb)
# written by Jane R. Rigby, which created narrowband maps of multiple emission lines in the Sunburst Arc,
# based upon various HST filter exposures. Unlike the previous file, the functionality of this file is 
# reduced and only creates Lya maps, and does not convolve copies of them to the F160W observations.
# Contact M. Riley Owens (m.riley.owens@gmail.com) to request additional functionality for other emission 
# lines or to restore the convolution procedures.

In [ ]:
import pandas 
import stsynphot
import synphot
from synphot.models import Empirical1D
import os
import warnings
import glob
import copy
import numpy as np
from astropy.convolution import convolve
from astropy.io import fits
from astropy import units as u
from astropy import wcs
import extinction

from regions import Regions

warnings.filterwarnings('ignore')
pandas.options.display.width = 0
pandas.set_option("display.max_columns", None)

In [3]:
def make():

    # Establish common directories
    home = os.getcwd()
    data = f'{home}/data'
    results = f'{home}/results'

    # The filepaths of the SEDs fitted to complete images of the entire Sunburst Arc galaxy
    SEDs = np.array([i for i in sorted(glob.glob(f'{data}/sed_fits/bestfit_model_image*_spec_filter6_cont.txt'))], dtype=str)

    # Create a blank dictionary
    df = {}
    
    # For each SED fit filepath
    for i, sed in enumerate(SEDs) :

        # Add the SED as a Pandas DataFrame to a dictionary with the filepath as its key
        df[sed] = pandas.DataFrame(np.transpose(np.loadtxt(sed)), columns=('Observed wavelength (Å)', 'Flux density (maggies)'))

    # ID of the SED to normalize the others against
    image = 3

    # Make an array that will store the normalization factors of the SEDs
    normfactors = np.zeros(shape=len(SEDs))

    # For each SED fit filepath
    for i, sed in enumerate(SEDs):

        # Determine the mean flux density (in Jy) between 1 and 1.05 microns
        fluxhere = (df[sed]['Flux density (maggies)'].loc[df[sed]['Observed wavelength (Å)'].between(1E4,1.05E4)]).mean() * 3631   
        normfactors[i] =  fluxhere

    # The scaling factor of each SED
    scaleby = normfactors / normfactors[image - 1]

    # Make an empty list that we will append spectra to
    arcspec = []

    # For each SED fit filepath
    for i, sed in enumerate(SEDs) :

        # Get the observed wavelength (Å) and flux density (maggies)
        w, f =  np.loadtxt(sed)

        # Convert the flux density to Janskys and scale the spectrum to match the given complete image.
        # This accounts for the different magnifications of the complete images. What is the purpose of
        # making the units Janskys?
        f_Jy = f * 3631 / scaleby[i]

        # Determine the Milky Way extinction according to Cardelli et al. (1989),
        # where E(B-V) = 0.09427 and R_v = 3.1. Also need to confirm how the E(B-V) 
        # is calculated. Original notebook states it follows Green et al. (2015) 
        # (ApJ, 810, 25), in mage_calc_MW_reddening.py.
        MW_extinction = extinction.ccm89(w, -1 * 3.1 * 0.09427, 3.1, unit='aa')

        # Deredden the flux densities
        f_Jy = f_Jy * 10**(-0.4 * MW_extinction)

        # Instantiate SourceSpectrum objects in synphot of the scaled and unscaled data and append them to separate lists
        arcspec.append(synphot.SourceSpectrum(Empirical1D, points=w * u.Angstrom, lookup_table=f_Jy * u.Jy))


    # Dictionary of the mean redshifts of the composite spectra of each stacked spectrum
    z = {
        'nonleaker' : 2.37060,
        'leaker'  : 2.37029
    }

    # Dictionary of rest wavelength segments (angstroms) free from prominent lines
    segments = np.array([
        [1250, 1260],
        [1265, 1275],
        [1310, 1320],
        [1350, 1360],
        [1420, 1430],
        [1450, 1460],
        [1470, 1480],
        [1490, 1500],
        [1510, 1520],
        [1560, 1630],
        [1680, 1715],
        [1720, 1740],
        [1760, 1810],
        [1820, 1850],
        [1970, 2000]
    ])

    # For the identifying label of each stacked MagE spectrum
    for i, label in enumerate(['nonleaker', 'leaker']):

        # Get the file path
        file = glob.glob(f'{data}/mage/sunburst_arc_{label}_stack_mage.txt')[0]

        # Get the wavelength bins and flux densities of the stacked spectrum
        w, f = np.loadtxt(file, delimiter='\t', comments='#', usecols=(0,1), unpack=True)

        # Determine the dispersion of the wavelength bins in the rest frame
        dispersion_rest = np.absolute(np.diff(w))

        # Compute the boxcar kernel that will convolve the data
        boxcar = int(np.ceil(100 / np.median(dispersion_rest)) // 2 * 2 + 1)

        f_masked = f

        # For each clean wavelength segment
        for j, segment in enumerate(segments):

            # Mask non-clean wavelength segments with NaNs
            f_masked = np.where((w >= segment[0]) & (w <= segment[1]), f_masked, np.nan)

        # Convolve the data with the boxcar kernel
        c = convolve(f_masked, np.ones((boxcar,)) / boxcar, boundary='fill', fill_value=np.nan)

        # Place the data in the observed frame
        w = w * (1 + z[label])
        f = f / (1 + z[label])

        # Replace Lya with the boxcar-smoothed estimate of the continuum
        f = np.where((w >= 4080) & (w <= 4125), c, f)
        
        # Replace any remaining NaNs with zeros
        f = np.where(np.isnan(f), 0, f)

        # Append the MagE spectra as synphot SourceSpectrum objects to the existing list of SourceSpectrum objects from
        # the SED fits
        arcspec.append(synphot.SourceSpectrum(Empirical1D, points=w * u.Angstrom, lookup_table=f * synphot.units.FLAM))

    # Make arrays of the filter information of the on- and off-band filters for the continuum
    # subtraction. These strings are in a specific format that synphot will recognize and use
    # to get instrumental information.
    on_setups  = np.array(['wfc3,uvis1,f410m'], dtype=str)
    off_setups = np.array(['wfc3,uvis1,f390w', 'wfc3,uvis1,f555w'], dtype=str)
    
    # Combine the two arrays of on- and off-band setups
    setups = np.append(on_setups, off_setups)

    # Make containers to store the results (consider changing variable name to n)
    n_spec = len(arcspec)# + len(magelabels)

    # The filters for Lya get 6 spectra (4 Gourav's + MagE leaker + Mage non-leaker)
    # The other filters get 4 spectra (Gourav's fits to each complete image)
    scaleF390W_for_Lya = np.empty(n_spec)  # Scale factor for continuum filter, before subtracting from filter w emiss line
    scaleF555W_for_Lya = np.empty(n_spec)  # Try F555W as cont filter instead of F390W
    contF410M = np.empty(n_spec)   # Calculated continuum in the filter containing the emission line
    contF410M_alt = np.empty(n_spec)   # Try F555W as cont filter instead of F390W

    # Treat Lya differently than the other filters, bc we have MagE spectra, and b/c of addnl cont uncertainty from Lya Forest
    allspec = copy.deepcopy(arcspec)

    # What is this meant to do?
    if len(allspec) != 6 : raise Exception("Copy of list of files was not deep")

    # For each synphot Source Spectrum object of the spectra
    for i, thisspec in enumerate(allspec):

        # Create a dictionary that will contain the count rates for each filter
        countrate = {}

        # For each filter
        for setup in setups :  # only F410M, F390W, F555W needed here

            # Get the bandpass and pass the spectrum through it as a synphot Observation object
            bp = stsynphot.band(setup)
            obs = synphot.Observation(thisspec, bp, force='taper')

            # Calculate the count rate measured from the filter-transmitted spectrum
            countrate[setup] = obs.countrate(stsynphot.conf.area).value

        # Compute the scaling factor for the F390W continuum estimate
        scaleF390W_for_Lya[i] = countrate['wfc3,uvis1,f410m'] / countrate['wfc3,uvis1,f390w']  # continuum for Lya
        
        # Why is the F410M continuum just the F410M count rate?
        contF410M[i] = scaleF390W_for_Lya[i] * countrate['wfc3,uvis1,f390w']
        
        # Compute the scaling factor for the F555W continuum estimate
        scaleF555W_for_Lya[i] = countrate['wfc3,uvis1,f410m'] / countrate['wfc3,uvis1,f555w']  # alt continuum for Lya

        # Why is the F410M continuum just the F410M count rate?
        contF410M_alt[i] = scaleF555W_for_Lya[i] * countrate['wfc3,uvis1,f555w']

    # Make arrays of the scale factors measured from each background treatment, 
    # as well as the corresponding means and standard deviations
    scalefactors = np.array([scaleF390W_for_Lya, scaleF555W_for_Lya])
    cont_scale_mean = np.array([np.mean(x) for x in scalefactors])
    cont_scale_std  = np.array([np.std(x) for x in scalefactors])

    # We need to subtract the sky background from the HST images, so let's get a
    # Region object from a .reg file representing a patch of the sky that is relatively
    # empty.
    region = Regions.read(f'{data}/box_for_median_imcoords_v4.reg', format='ds9')[0]

    # For convenience, get the headers of the v4 and v5 data. Since the WCS should be the same across all data of the same
    # version, it shouldn't (?) matter which filter we pick from either version, so just grab what we have laying around.
    v4_header = fits.getheader(f'{data}/hst/V4.0_PSZ1G311.65-18.48_F275W_0.03g0.8_cr2.5_0.7_drc_sci.fits')
    v5_header = fits.getheader(f'{data}/hst/V5.0_PSZ1G311.65-18.48_F410M_0.03g0.6_crsc1.2_0.7crsn3.5_3.0_drc_sci.fits')

    # The .reg file was originally created (by Jane Rigby, I presume?) from the pixel coordinates of the v4 reduction of the Sunburst 
    # Arc HST data. Since then, v5 is now available and should be used instead of v4. Since v5 has a slightly different WCS and the 
    # .reg file is built from pixel coordinates of the old WCS, we need to convert it to sky coordinates based on the WCS of the v4
    # data.
    sky_region = region.to_sky(wcs.WCS(v4_header))

    # For convenience for future users, make a copy of the region file in sky coordinates.
    # I am dropping the 'vX' suffix since this file is not built from pixel coordinates of
    # a certion reduction version of data.
    sky_region.write(f'{home}/results/box_for_median_imcoords.reg', format='ds9', overwrite=True)

    # Now get the region as pixel coordinates in the v5 WCS. We'll use this to create a mask of the region.
    pixel_region = sky_region.to_pixel(wcs.WCS(v5_header))

    # Make a mask of the region and save it
    mask = pixel_region.to_mask().to_image((v5_header['NAXIS2'], v5_header['NAXIS1']))
    fits.writeto(f'{home}/results/masks/box_for_median_imcoords_mask_v5.fits', mask.data, header=v5_header, overwrite=True)

    # For each filter used as a continuum estimate
    for i, filter in enumerate(['F390W', 'F555W']):

        # Get the data and headers of the on- and off-band filters
        data_off, header_off = fits.getdata(f'{data}/hst/V5.0_PSZ1G311.65-18.48_{filter}_0.03g0.6_crsc1.2_0.7crsn3.5_3.0_drc_sci.fits', header=True)
        data_on, header_on   = fits.getdata(f'{data}/hst/V5.0_PSZ1G311.65-18.48_F410M_0.03g0.6_crsc1.2_0.7crsn3.5_3.0_drc_sci.fits', header=True)

        # Compute the median values of the masked sky patch
        med_val_off = np.nanmedian(data_off * mask.data)
        med_val_on = np.nanmedian(data_on * mask.data)

        # Continuum subtract the on-band data with the off-band continuum estimate
        scale_this_offband = cont_scale_mean[i]
        data_cont = (data_off - med_val_off) * scale_this_offband  # remove median sky bkg
        data_cont_sub = (data_on - med_val_on) - data_cont # remove median sky bkg
        cont_math = f'F410M - {filter} * (CNTSUBT)'

        # Propagate the uncertainty to the final continuum-subtracted image
        n = np.sqrt((np.nanstd(data_on * mask.data))**2 +
            (np.nanstd(data_on * mask.data))**2 + 
            ( (data_off - med_val_off) * cont_scale_mean[i] * np.sqrt( (np.sqrt((np.nanstd(data_off * mask.data))**2 + (np.nanstd(data_off * mask.data))**2) / (data_off - med_val_off))**2 + (cont_scale_std[i] / cont_scale_mean[i])**2 ) )**2
        )

        # Initiate headers for the two extensions of the new file from the WCS
        # it will use. We will then build on the headers from there.
        header_f = wcs.WCS(header_on).to_header()
        header_n = wcs.WCS(header_on).to_header()

        # Construct the CD matrix of the WCS system from the on-band header
        cd_matrix = wcs.WCS(header_on).wcs.cd

        # For the headers of the data and uncertainty extensions
        for j, header in enumerate([header_f, header_n]):

            # Set the CD matrix keywords in the new headers of the saved Lya maps
            header['CD1_1'] = cd_matrix[0,0]
            header['CD1_2'] = cd_matrix[0,1]
            header['CD2_1'] = cd_matrix[1,0]
            header['CD2_2'] = cd_matrix[1,1]

            # Add various informational keywords to the header
            header.insert(1, ('CREATOR', 'M. Riley Owens', 'Creator of this file'))
            header.insert(2, ('CREATOR_EMAIL', 'm.riley.owens@gmail.com', 'Email of the creator of this file'))
            header['GENFILE'] = ('nb.ipynb', 'The file used to create this file')
            header['FILENAME'] = (f'Lya_cont_sub_{filter}.fits', 'Name of file')
            header['ON_FILE'] = ('V5.0_PSZ1G311.65-18.48_F410M_0.03g0.6_crsc1.2_0.7crsn3.5_3.0_drc_sci.fits', 'The name of the on-band file')
            header['OFF_FILE'] = (f'V5.0_PSZ1G311.65-18.48_{filter}_0.03g0.6_crsc1.2_0.7crsn3.5_3.0_drc_sci.fits', 'The name of the off-band file')
            header['CNTMATH'] = (f'F410M - {filter} * SCALE', 'Formula used to calculate the Lyman-alpha emission in the narrowband image')
            header['SCALE'] = (cont_scale_mean[i], 'The mean continuum scaling factor, as inferred from the distribution of scaling factors derived from the 4 SED fits and 2 stacked MagE spectra')
            header['SCALEU'] = (cont_scale_std[i], 'The standard deviation of the continuum scaling factor, as inferred from the distribution of scaling factors derived from the 4 SED fits and 2 stacked MagE spectra')
            
            # For each keyword in an array of useful keywords that the observations share
            for k, keyword in enumerate(np.array(['FILETYPE', 'TELESCOP', 'INSTRUME', 'EQUINOX', 'IMAGETYP',
                'PRIMESI', 'OBSTYPE', 'BUNIT', ], dtype=str)):

                # Add the keyword to the header
                header[keyword] = (header_on[keyword], header_on.comments[keyword])

        # Create a HDU list from the data and uncertainties and the constructed headers
        hdu_f = fits.PrimaryHDU(data_cont_sub, header_f, 'DATA')
        hdu_n = fits.ImageHDU(n, header_n, 'UNCERTAINTY')
        hdul = fits.HDUList([hdu_f, hdu_n])

        # Save the Lya map
        hdul.writeto(f'{results}/lya_maps/Lya_cont_sub_{filter}.fits', overwrite=True)

In [4]:
make()